# Test 1

In [ ]:
import mib as mb1
import mib_v2_3_1 as mb
import matplotlib.pyplot as plt
import time
import tracemalloc

## Planteamiento

Comprobar del rendimiento del motor de inferencia con problema pequeño y con tres eventon con distribuciones uniformes, donde su cardinalidad de los tres va de uno a cien.

### Variables

- $A = \{0, 2, ..., n-1\}$

- $B = \{0, 2, ..., n-1\}$

- $C = \{0, 2, ..., n-1\}$

### Modelo de descomposición

$P(ABC) = P(A)P(B|A)P(C|AB)$.

$\;\;\;\;\;\;\;\;\;\;\;\;$<img src="./P(ABC).jpg" width=200/>

### Formas 

$P(A)$, $P(B|A)$ y $P(C|AB)$ serán distribuciones uniformes.

## Test

### MIB1.0

In [ ]:
n = 10
timesM1 = []
memoryM1 = []

timesC1 = []
memoryC1 = []

for i in range(n):
    Events = [x for x in range(i+1)]
    # Varaibles
    A = mb1.Var('A',Events)
    B = mb1.Var('B',Events)
    C = mb1.Var('C',Events)

    dA = {}
    dB_A = {}
    dC_AB = []
    value = 1 / (i+1)
    for e in Events:
        dA[e] = value
        dB_A[e] = {}
        for e1 in Events:
            dB_A[e][e1] = value
            tC_AB = []
            for e2 in Events:
                tC_AB.append(value)
            dC_AB.append(tuple(tC_AB))
    
    PA=mb1.Distrib(name='P(A)',variable=[A],tabla=dA)
    PB_A=mb1.DistribCond(name='P(B|A)',var=B,indep=[A],tabla=dB_A)
    PC_AB=mb1.DistribCond('P(C|AB)',C,[A,B],dC_AB)
    
    PABC=mb1.JointDistrib(name='P(ABC)',variables=[A,B,C],descomp=[PA,PB_A,PC_AB])
    Q_ABC=mb1.Question(joint=PABC)

    #Inferencia de P(B)
    tracemalloc.start()
    
    inicio = time.time()
    PB = Q_ABC.query(searched=[B])
    fin = time.time()
    
    memoria_actual, memoria_pico = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    tiempo_ejecucion = fin - inicio
    
    timesM1.append(tiempo_ejecucion)
    memoryM1.append(memoria_pico)
    
    #Inferencia de P(A|BC)
    tracemalloc.start()
    
    inicio = time.time()
    PA_BC = Q_ABC.query(searched=[A], known=[B,C])
    fin = time.time() 
    
    memoria_actual, memoria_pico = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    tiempo_ejecucion = fin - inicio
    
    timesC1.append(tiempo_ejecucion)
    memoryC1.append(memoria_pico)

### MIB2.0

In [ ]:
n = 10
timesM = []
memoryM = []

timesC = []
memoryC = []

for i in range(n):
    Events = set([x for x in range(i+1)])
    
    # Generación de distribuciones de probabilidad (Distribuciones uniformes).
    dA = {}
    dB_A = {}
    dC_AB = {}
    value = 1 / (i+1)
    for x in range(i+1):
        dA[x] = value
        dB_A[(x,)] = {}
        for y in range(i+1):
            dB_A[(x,)][y] = value
            dC_AB[(x,y)] = {}
            for z in range(i+1):
                dC_AB[(x,y)][z] = value
    
    # Variables para el motor de inferencia.
    A = mb.Var('A', Events)
    PA = mb.Distrib(dA, (A.getName(),))
    B = mb.Var('B', Events)
    PB_A = mb.CondDistrib(dB_A, (B.getName(),), (A.getName()))
    C = mb.Var('C', Events)
    PC_AB = mb.CondDistrib(dC_AB, (C.getName(),), (A.getName(),B.getName()))
    
    # Probabilidad conjunta (Espesificación)
    PABC = mb.Specification(set([A,B,C]), set([PA, PB_A, PC_AB]))

    Q_ABC = mb.Question(PABC)
    
    # Inferencia de una marginal
    tracemalloc.start()
    
    inicio = time.time()
    PB = Q_ABC.DistributionQuery(set([B]))
    fin = time.time()
    
    memoria_actual, memoria_pico = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    tiempo_ejecucion = fin - inicio
    
    timesM.append(tiempo_ejecucion)
    memoryM.append(memoria_pico)
    
    # Inferencia de una condicional dada observaciones
    tracemalloc.start()
    
    inicio = time.time()
    PA_BC = Q_ABC.DistributionQuery(set([A]), set([B,C]))
    fin = time.time()
    
    memoria_actual, memoria_pico = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    tiempo_ejecucion = fin - inicio
    
    timesC.append(tiempo_ejecucion)
    memoryC.append(memoria_pico)

## Datos obtenidos

### Tiempos

#### Tiempos de inferencia para $P(B)$

In [ ]:
xn = [i+1 for i in range(n)]
fig, ax = plt.subplots(1, 2)
# fig.set_size_inches(15,5)
# MIB_1.0
ax[0].plot(xn, timesM1, color='b',label = 'MIB_1.0')
ax[0].set_title("Tiempo de inferencia para P(B) (MIB_1.0)")
ax[0].set_xlabel("n")
ax[0].set_ylabel("segundos")

# MIB_2.0
ax[1].plot(xn, timesM, color='r',label = 'MIB_2.0')
ax[1].set_title("Tiempo de inferencia para P(B) (MIB_2.0)")
ax[1].set_xlabel("n")
ax[1].set_ylabel("segundos")

# Mostrar las gráficas
plt.show()

In [ ]:
fig, ax = plt.subplots()

# MIB_1.0
ax.plot(xn, timesM1, color='b', label = 'MIB_1.0')

# MIB_2.0
ax.plot(xn, timesM, color='r',label = 'MIB_2.0')
ax.set_title("Tiempo de inferencia para P(B)")

ax.set_xlabel("n")
ax.set_ylabel("segundos")
ax.legend(loc = 'upper left')

# Mostrar las gráficas
plt.show()

#### Tiempos de inferencia para $P(C|AB)$

In [ ]:
xn = [i+1 for i in range(n)]
fig, ax = plt.subplots(1, 2)
# fig.set_size_inches(15,5)
# MIB_1.0
ax[0].plot(xn, timesC1, color='b',label = 'MIB_1.0')
ax[0].set_title("Tiempo de inferencia para P(C|AB) (MIB_1.0)")
ax[0].set_xlabel("n")
ax[0].set_ylabel("segundos")

# MIB_2.0
ax[1].plot(xn, timesC, color='r',label = 'MIB_2.0')
ax[1].set_title("Tiempo de inferencia para P(C|AB) (MIB_2.0)")
ax[1].set_xlabel("n")
ax[1].set_ylabel("segundos")

# Mostrar las gráficas
plt.show()

In [ ]:
fig, ax = plt.subplots()

# MIB_1.0
ax.plot(xn, timesC1, color='b', label = 'MIB_1.0')

# MIB_2.0
ax.plot(xn, timesC, color='r',label = 'MIB_2.0')
ax.set_title("Tiempo de inferencia para P(B)")

ax.set_xlabel("n")
ax.set_ylabel("segundos")
ax.legend(loc = 'upper left')

# Mostrar las gráficas
plt.show()

### Memoria

#### Memoria para la inferencia de P(B)

In [ ]:
fig, ax = plt.subplots(1, 2)
# fig.set_size_inches(15,5)
# MIB_1.0
ax[0].plot(xn, memoryM1, color='b',label = 'MIB_1.0')
ax[0].set_title("Memoria para la inferencia para P(B) (MIB1.0)")
ax[0].set_xlabel("n")
ax[0].set_ylabel("MB")

# MIB_2.0
ax[1].plot(xn, memoryM, color='r',label = 'MIB_2.0')
ax[1].set_title("Memoria para la inferencia para P(B) (MIB2.0)")
ax[1].set_xlabel("n")
ax[1].set_ylabel("MB")

# Mostrar las gráficas
plt.show()

In [ ]:
fig, ax = plt.subplots()

# MIB_1.0
ax.plot(xn, memoryM1, color='b',label = 'MIB_1.0')

# MIB_2.0
ax.plot(xn, memoryM, color='r',label = 'MIB_2.0')
ax.set_title("Memoria para la inferencia para P(B)")

ax.set_xlabel("n")
ax.set_ylabel("segundos")
ax.legend(loc = 'upper left')

# Mostrar las gráficas
plt.show()

#### Memoria para la inferencia de P(C|AB)

In [ ]:
fig, ax = plt.subplots(1, 2)
# fig.set_size_inches(15,5)
# MIB_1.0
ax[0].plot(xn, memoryC1, color='b',label = 'MIB_1.0')
ax[0].set_title("Memoria para la inferencia para P(C|AB) (MIB1.0)")
ax[0].set_xlabel("n")
ax[0].set_ylabel("MB")

# MIB_2.0
ax[1].plot(xn, memoryC, color='r',label = 'MIB_2.0')
ax[1].set_title("Memoria para la inferencia para P(C|AB) (MIB2.0)")
ax[1].set_xlabel("n")
ax[1].set_ylabel("MB")

# Mostrar las gráficas
plt.show()

In [ ]:
fig, ax = plt.subplots()

# MIB_1.0
ax.plot(xn, memoryC1, color='r',label = 'MIB_1.0')
# MIB_2.0
ax.plot(xn, memoryC, color='b',label = 'MIB_2.0')

ax.set_title("Memoria para la inferencia para P(C|AB)")

ax.set_xlabel("n")
ax.set_ylabel("MB")
ax.legend(loc = 'upper left')

# Mostrar las gráficas
plt.show()

# Test 2

In [ ]:
import mib as mb1
import mib_v2_3_1 as mb
import time

## Planteamineto

Se tiene un conjunto de correos, en los cuales se sabe si son spam o no, y se quiere saber si dada las palabras presentes en el correo el correo pertenece a spam.

### Variables

* $Spam := \{0,1\}$
* $W_i = \{0,1\}$

* $P(Spam = 0) = 0.25$
* $P(Spam = 1) = 0.75$

### Modelo de descomposición

$P(Spam W_0 W_1 ... W_n ) = P(Spam)\prod_i P(W_i|Spam)$.

Cada una de las $n$ formas de $P(W_i|Spam)$ debe ser espacificado. Para el conteo de la i-ésima palabra que aparesca spam y no spam 

* $P(W_i|Spam)$
    * $P(W_i = true | Spam = false) = \frac{1 + {n^i}_f}{|W_i| + n_f}$
    * $P(W_i = true | Spam = true) = \frac{1 + {n^i}_v}{|W_i| + n_t}$

donde, ${n^i}_f$ es el número de apariciones de la i-ésima palabra en correos que no son spam y $n_f$ es el número total de correos que no son spam, ${n^i}_v$ es el número de apariciones de la i-ésima palabra en correos que son spam y $n_v$ es el número total de correos que son spam.

## Implementación 

### Generación de distribuciones

#### $P(Spam)$

In [ ]:
dSpam = {(0,):0.25, (1,):0.75}
# Variables para el motor de inferencia.
Spam = mb.Var('Spam',set([0,1]))
PSpam = mb.Distrib(dSpam, (Spam.getName(),))
PSpam.print_table()

#### Variales $W_i$

In [ ]:
vocabulario = ['fortune','next','programming','money','you']
W = {}
events = set([0,1])

for palabra in vocabulario:
    W[palabra] = mb.Var(palabra, events)
    print(f'W[{palabra}]: {W[palabra]}')

In [ ]:
nif = [0,125,250,0,125] # Canitdad de veces que aparece una palabra en correos que no son spam.
niv = [375,0,0,750,375] # Canitdad de veces que aparece una palabra en correos que son spam.

nf = 250
nv = 750

PWi_Spam = {}

i = 0
for Wi in W:
    tf = (1 + nif[i]) / (2 + nf)
    tt = (1 + niv[i]) / (2 + nv)
    dWi_Spam = {(0,): {(0,): 1 - tf, (1,):  tf},
                (1,): {(0,): 1 - tt, (1,): tt}}
    
    PWi_Spam[Wi] = mb.CondDistrib(dWi_Spam, (W[Wi].getName(),), (Spam.getName(),))
    i += 1
for Wi in W:   
    PWi_Spam[Wi].print_table()

### Generación de pregunta

In [ ]:
vars = set([W[i] for i in W] + [Spam])
distribs = set([PWi_Spam[i] for i in W] + [PSpam])
P_WiSpam = mb.Specification(vars, distribs)
Q_WiSpam = mb.Question(P_WiSpam)

### Consulta

In [ ]:
inicio = time.time()
PSpam_wn = Q_WiSpam.DistributionQuery(set([Spam]), set([W[wi] for wi in W]))
fin = time.time()
tiempo_ejecucion = fin - inicio
PSpam_wn.print_table()

In [ ]:
tiempo_ejecucion

# Test 3

In [ ]:
import mib as mb1
import mib_v2_3_1 as mb

## Planteamiento

Una compañía sabe que la contratación profesional de un candidato se efectúa luego de una entrevista, que se lleva a cabo en función de las calificaciones reportadas en el certificado del candidato y si éste tiene experiencia laboral o no. Generalmente, sólo se consideran los candidatos con calificaciones sobresalientes o regulares, y la entrevista arroja típicamente tres tipos de apreciación.

La compañía ha elaborado el siguiente modelo de contratación:

$\;\;\;\;\;\;\;\;\;\;\;\;$<img src="./MODELO_EMPRESA.jpg" width=300/>

La compañía ha recolectado datos históricos de los últimos 5 años sobre la evaluación de 500 candidatos y sabe que la proporción de candidatos con calificaciones regulares es del 30%, mientras que los candidatos con experiencia laboral representan el 60%. En la mejor condición, con calificaciones sobresalientes y experiencia laboral, los candidatos obtienen la más alta apreciación en la entrevista en un 80%, y la peor apreciación en un 2% de los casos. Estos porcentajes cambian respectivamente a 30% y 10% cuando las calificaciones no son las mejores, pero sí hay experiencia, y ambas son del 30% con calificaciones sobresalientes pero sin experiencia. En el peor caso, con calificaciones regulares y sin experiencia, los candidatos obtienen la mejor apreciación en la entrevista en un 10% y la peor en un 70%. Por último, se sabe que la tasa de candidatos rechazados es de un 10% cuando obtienen una apreciación favorable, un 60% cuando obtienen una apreciación regular, y un 99% cuando obtienen una apreciación desfavorable en la entrevista.

### Variables

* $C := \{0,1\}$, donde C representa las calificaciones; 0 si son calificaciones reguales, 1 otro caso.
* $E : = \{0,1\}$, donde E representa la experencia; 
* $N : = \{0,1,2\}$, donde  
* $O : = \{0,1\}$, donde

### Modelo de descomposición

$P(C E N O) = P(C)P(E)P(N|C E)P(N|O)$

## Implementación

In [ ]:
E = mb.Var('E', set([0,1]))
C = mb.Var('C', set([0,1]))
O = mb.Var('O', set([0,1]))
N = mb.Var('N', set([0,1,2]))

dE = {(0,):0.4, (1,):0.6}
dC = {(0,):0.3, (1,):0.7}
dN_EC = {(0,0):{(0,):0.7, (1,):0.2, (2,):0.1},
        (0,1):{(0,):0.3, (1,):0.4, (2,):0.3},
        (1,0):{(0,):0.1, (1,):0.6, (2,):0.3},
        (1,1):{(0,):0.02, (1,):0.18, (2,):0.8}}
dO_N={(0,):{(0,):0.99, (1,):0.01},
      (1,):{(0,):0.6, (1,):0.4},
      (2,):{(0,):0.1, (1,):0.9}}

P_E = mb.Distrib(dE, (E.getName(),))
P_C = mb.Distrib(dC, (C.getName()))
P_N_EC = mb.CondDistrib(dN_EC, (N.getName(),), (E.getName(), C.getName()))
P_O_N = mb.CondDistrib(dO_N, (O.getName(),), (N.getName()))

### Creación de la especificación

In [ ]:
vars = set([E,C,N,O])
P_ECNO = mb.Specification(vars,set([P_E,P_C,P_N_EC,P_O_N]))
Q_P = mb.Question(P_ECNO)

### Preguntas

In [ ]:
#1. Cuál es su tasa de contratación 
#(i.e. cuál es la probabilidad de ser contratado). P(O)
CONT = Q_P.Query(vars=(O,), vars_values = (1,))
print(f'{round(CONT*100,2)}%')

In [ ]:
#2. Cuantos candidatos ha contratado la compañía en los últimos 5 años. 
#X=P(O=0)*500.
from math import floor

X = floor(500*CONT)
print(X)

In [ ]:
#3. Cuál es la tasa de apreciaciones favorables en la compañía.
PN = Q_P.DistributionQuery(set(N))
PN.print_table()
FAV = Q_P.Query(vars=(N,), vars_values = (2,))
print(f'{FAV*100}%')

In [ ]:
#4. Cuál es la tasa de apreciaciones regulares dado que se contrata a alguien. 
TAF = Q_P.Query(vars=(N,), indep=(O,), vars_values = (1,), indep_values=(0,))
print(f'{TAF*100}%')

In [ ]:
#5. Cuál es la tasa de apreciaciones regulares dado que se contrata a alguien.
TAF = Q_P.Query(vars=(N,), indep=(O,), vars_values = (0,), indep_values=(0,))
print(f'{int(round(TAF*X,0))}')

In [ ]:
#6. Cuantos de los candidatos contratados obtuvieron una apreciación regular. 
#P(N=1|O=0)*X
print(floor(TAF*X))

In [ ]:
#7. Cómo se distribuye la contratación de candidatos en función de sus calificaciones. 

PO_C = Q_P.DistributionQuery(set([O]),set([C]))
PO_C.table

In [ ]:
#8. Cómo se distribuye la contratación de candidatos en función de su experiencia. 

PO_E = Q_P.DistributionQuery(set([O]),set([E]))
PO_E.table

In [ ]:
#9. Cuál es la probabilidad de que alguien con experiencia laboral tenga calificaciones sobresalientes. 
#P(C=0|E=0)
ECS = Q_P.Query(vars=(C,), indep=(E,), vars_values = (0,), indep_values=(0,))
print(f'{ECS*100}%')

In [ ]:
#10. Cuál es la distribución conjunta del personal contratado y 
#la apreciación de la entrevista.
PON = Q_P.DistributionQuery(set([N,O]))
PON.table

# Test 4

In [ ]:
import tpproc as tp
from collections import Counter  #regresa un diccionario con conteos
import glob
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from collections import OrderedDict #diccionarios ordenados
import numpy as np
import mib as mb1
import mib_v2_3_1 as mb

## Planteamiento

**Problema de autoria de textos**

## Especificación 

### Variables
- $A$ : Variable para los autores
- $T$ : Variable para los tipos de textos
- $W_I$ : Variable para las palabras

**Variable $A$**

- $A = $ Alfonso Reyes
- $A = $ Esopo
- $A = $ Fuentes
- $A = $ García Márquez
- $A = $ Gibran
- $A = $ La Fontanie
- $A = $ Onetti
- $A = $ Quiroga
- $A = $ Rulfo
- $A = $ Saramago

**Variable $T$**

- $T = $ minicuento
- $T = $ cuento
- $T = $ fábula

**Variable $W_i$**

- $W_i = \{0,1\}$, donde cada $W_i$ representa una variable para una palabra y sus representaciones númericas son las siguientes:
- $W_i = 0$ -> la palabra no esta.
- $W_i = 1$ -> la palabra si esta.

### Suponiendo un modelo de descomposición exacta:

$P(A)P(T|A)\displaystyle \prod_{i=0}^{n} P(W_i|AT)$

## Implementación

### Obtención de datos para el test

In [ ]:
archivos = glob.glob('./Train/*/*')
archivos,nombres = tp.carga_cuentos(archivos)
df_train = tp.lee_cuentos(archivos)
df_train.head()

In [ ]:
archivos = glob.glob('./Test/*')
archivos,nombres = tp.carga_cuentos(archivos)
df_test = tp.lee_cuentos(archivos,test=True)
df_test.head()

#### Data Frames de trabajo

In [ ]:
s = df_train.texto.str.cat(sep=' ').split()
print(len(s))
conteos=Counter(s)
nuevo_vocabulario = list(set([palabra for palabra in s if conteos[palabra] <= 20 and conteos[palabra] >= 10 ]))
print(len(nuevo_vocabulario))

In [ ]:
df_train_T = df_train.copy()
df_train_T['nuevo_texto'] = df_train_T.texto.str.split().\
    apply(lambda texto: [w for w in texto if w in nuevo_vocabulario]).\
    apply(lambda x : ' '.join(x))
df_train_T['Conteos']=df_train_T.nuevo_texto.str.split().apply(Counter)


df_train_T.head()

In [ ]:
s=df_test.texto.str.cat(sep=' ').split()
print(len(s))
conteos=Counter(s)
nuevo_vocab = list(set([palabra for palabra in s if conteos[palabra] <= 20 and conteos[palabra] >= 10]))
print(len(nuevo_vocab))

In [ ]:
df_test_T = df_test.copy()
df_test_T['nuevo_texto']=df_test_T.texto.str.split().\
    apply(lambda texto: [w for w in texto if w in nuevo_vocab]).\
    apply(lambda x : ' '.join(x))
df_test_T['nuevo_total']=df_test_T.nuevo_texto.str.split().apply(len)
df_test_T['Conteos']=df_test_T.nuevo_texto.str.split().apply(Counter)


df_test_T.head()

#### Tamaño del vocabulario conjunto (total de palabras)

In [ ]:
vocabulario = list(set(nuevo_vocabulario+nuevo_vocab))
print(len(vocabulario))

In [ ]:
voc_dict = OrderedDict(zip(vocabulario,range(len(vocabulario))))
v = list(voc_dict.keys())

#### Conteos

In [ ]:
# Ocurrencias de cada autor y conteos de ocurrencias y de número de autores
oc_autor = Counter(df_train_T.autor)
tot_oc_aut = np.sum(list(oc_autor.values()))
tot_autores = len(oc_autor)

# Ocurrencias de cada tipo y conteos de ocurrencias y de número de tipos
oc_tipo = Counter(df_train_T.tipo)
tot_oc_tipo = np.sum(list(oc_tipo.values()))
tot_tipos = len(oc_tipo)

### Creación de variables

In [ ]:
# Autor
autor_val = dict(zip(oc_autor.keys(),range(len(oc_autor.keys()))))
val_autor = dict(zip(range(len(oc_autor.keys())),oc_autor.keys()))
A = mb.Var('A',set(oc_autor.keys()))
# Tipos
tipo_val = dict(zip(oc_tipo.keys(),range(len(oc_tipo.keys()))))
val_tipo = dict(zip(range(len(oc_tipo.keys())),oc_tipo.keys()))
T = mb.Var('T',set(oc_tipo.keys()))

# Palabras
W = {}
for w in voc_dict:    # vocabulario reducido
    W[w] = mb.Var(w,set([0,1]))

### Identificación de parámetros (estadísticas)

### Método:

1. Descomposición exacta: $P(A)P(T|A)P(W|AT)$
2. Calcular las estadísticas por cada distribución de la descomposición
3. Para la variable Autor $P(A)$:
    - 3.1 Contar el número total de ocurrencias de autores en Train: $n_a$.
    - 3.2 Contar el número de ocurrencias de cada autor en Train: $n^i$.
    - 3.3 Dividir este número entre el número total de ocurrencias de autores en Train: $\displaystyle \frac{n^i}{n_a}$.
4. Para la variable Tipo $P(T|A)$:
    - 4.1 Obtener el conjunto de autores (valores únicos) $a$; entradas a la Tabla.
    - 4.2 Obtener el conjunto de tipos (valores únicos) $j$; estas son las salidas de la Tabla.
    - 4.3 Calcular el producto cartesiano de las combinaciones posibles (a,j).
    - 4.4 Contar el número de ocurrencias totales de cada $j$ por cada autor: $n^j_a$.
    - 4.5 Dividir este número entre el número total de tipos por autor: $\displaystyle \frac{n^j_a}{\sum n^j_a}$.
    - 4.6 Aplicar la corrección de Laplace para evitar probabilidades en $0$.
5. Para las variables W (palabras) $P(W|AT)$: **NOTA** se trata de calcular N Tablas, donde N es el tamaño del vocabulario.
    - 5.1 Calcular el producto cartesiano de las combinaciones posibles (a,j); estas son las entradas a cada Tabla.
    - 5.2 Por cada combinación, calcular la probabilidad de cada palabra del vocabulario, en función de su ocurrencia en esa combinación, aplicando la corrección de Laplace. 

##### Distribución P(A)

- Contar el número total de ocurrencias de autores en Train: $n_a$: _tot_oc_aut_
- Contar el número de ocurrencias de cada autor en Train: $n^i$: _oc_autor_ 
- Dividir este número entre el número total de ocurrencias de autores en Train: $\displaystyle \frac{n^i}{n_a}$

In [ ]:
n_= []
for autor in oc_autor:
    n_.append(((autor,),oc_autor[autor]/tot_oc_aut))

#Dicionario de valores de probabilidad
dA = dict(n_)
#Distribución de probabilidad
PA = mb.Distrib(table = dA, columns=('A',))
print(A.getValues())
PA.print_table()

#### Distribución P(T|A)

   - Obtener el conjunto de autores (valores únicos) $a$; entradas a la Tabla: _df_train.autor_
   - Obtener el conjunto de tipos (valores únicos) $j$; estas son las salidas de la Tabla: _df_train.tipo_ 
   - Calcular el producto cartesiano de las combinaciones posibles (a,j): _pares_ta_
   - Contar el número de ocurrencias totales de cada $j$ por cada autor: $n^j_a$: _conteo_pares_
   - Dividir este número entre el número total de tipos por autor: $\displaystyle \frac{n^j_a}{\sum n^j_a}$.
   - Aplicar la corrección de Laplace para evitar probabilidades en $0$.

In [ ]:
# Ocurrencias (conteos) de cada combinación (tipo,autor)
conteo_pares = Counter(zip(df_train_T.autor,df_train_T.tipo))
# print(conteo_pares)

# Combinaciones (tipo,autor)
autores = list(set(df_train_T.autor))
tipos = list(set(df_train_T.tipo))

dT_A = {}

for autor, tipo in product(*[autores,tipos]):
    par = (autor, tipo)
    ak = (autor,)
    
    if ak in dT_A.keys():
        if par in conteo_pares.keys():
            dT_A[ak][(tipo,)] = conteo_pares[par]
        else:
            dT_A[ak][(tipo,)] = 0 
    else:
        if par in conteo_pares.keys():
            dT_A[ak] = {(tipo,): conteo_pares[par]}
        else:
            dT_A[ak] = {(tipo,): 0}


# Corrección de Laplace en el caso general
# Se corrige la misma tabla que se senvía como argumento; no se crea una nueva.
def Laplace_gral(tabla):
    n = len(tabla[list(tabla.keys())[0]]) 
    for k in tabla.keys():
        registro = tabla[k]
        n_j = np.sum(list(registro.values()))
        for i in registro.keys():
            pb = (registro[i] + 1)/(n_j+n)
            tabla[k][i]=pb
    return

Laplace_gral(dT_A)

PT_A = mb.CondDistrib(set([T]), set([A]), dT_A, (T.getName(),), (A.getName(),))

In [ ]:
PT_A.print_table()

#### $P(W|AT)$

   - Calcular el producto cartesiano de las combinaciones posibles (a,j); estas son las entradas a cada Tabla.
   - Por cada combinación, calcular la probabilidad de cada palabra del vocabulario, en función de su ocurrencia en esa combinación, aplicando la corrección de Laplac

* Las palabras a tomar en cuenta son las del vocabulario.
* Se deben codificar las combinaciones (a,t) en valores numéricos.
* Por cada combinación (a,t):
    - si existe una historia (nuevo_texto), entonces cada palabra de este cuento está presente, y el resto de las palabras del vocabulario están ausentes.
    - si no hay historia, entonces ninguna palabra del vocabulario está presente --> Probabilidad 0 para todo el vocabulario.
* Al aplicar la corrección de Laplace, evitamos que una combinación (a,t), que por ahora no tiene texto, se considere imposible, al asignarle una probabilidad mínima a cada palabra del vocabulario.
* Para facilitar el "conteo" (presencia/ausencia) haremos un diccionario del vocabulario, incialmente en ceros

In [ ]:
# Combinaciones (tipo,autor)
autores = list(set(df_train_T.autor))
tipos = list(set(df_train_T.tipo))
pares_ta = list(product(autores,tipos))

# Conteos de palabras en nuevo_texto por pares (a,t) en el data frame de train
conteo_w = dict(df_train_T.Conteos)
pares_train = list(zip(df_train_T.autor,df_train_T.tipo))

for i,k in enumerate(conteo_w.keys()):
    conteo_w[k] = {pares_train[i] : dict(conteo_w[k])}
    
def check(palabra,pares_ta):
    pw_ = {}
    var = W[palabra]
    
    for par in pares_ta:
        cw_1 = 0   # conteos de presencia 
        cw_0 = 0   # y ausencia en 0
        it = 0     # total de pares (a,t) contabilizados
        
        for item in list(conteo_w.values()):  # checamos cada par (a,t) en el conjunto de train
            # print(i,list(item.keys())[0], end=' ')
            it += 1
            if par in item: # checamos si el par (a,t) de entrada tiene correspondencia en Train
                if palabra in list(item.values())[0]: #checamos si la palabra existe en el par (a,t) de Train
                    cw_1 += 1
                else:
                    cw_0 += 1  #debemos contar también la no existencia por si hay igualdad
        
        if cw_0 == cw_1: # la palabra aparece por igual en todos los pares Train, o bien, no hay pares Train
            pw_1 = 0.5   #corrección de Laplace que indica que la palabra puede o no estar por igual
            pw_0 = 0.5
            
        else:
            pw_1=(1+cw_1)/(len(var.getValues())+it) #corrección de Laplace en caso de que la palabra exista
            pw_0 = 1-pw_1
        
        pw_[par] = {(0,): pw_0, (1,): pw_1}
    pw_ = OrderedDict(sorted(pw_.items()))
    return pw_
        
    
PW_AT = {}
for w in voc_dict:    # vocabulario reducido
    t = check(w,pares_ta)
    PW_AT[w] = mb.CondDistrib(set([W[w]]),set([A,T]), dict(t), (W[w].getName(),), (A.getName(), T.getName()))

In [ ]:
palabra = list(PW_AT.keys())[0]
print(palabra+':')
PW_AT[list(PW_AT.keys())[0]].print_table()

In [ ]:
vl =  [A,T] + [W[w] for w in W] 
dl = [PA,PT_A] + [PW_AT[w] for w in PW_AT]
vars_set = set(vl)
dist_set = set(dl)

## Pregutas 

In [ ]:
PATWi = mb.Specification(vars_set, dist_set)
QW_AT = mb.Question(PATWi)

### 1. ¿Quién escribió "El caballo y el Lobo"?

In [ ]:
dcont = df_test_T.loc[df_test['titulo'] == 'el caballo y el lobo', 'Conteos'].iloc[0]
wit = []
for w in dcont:
    wit.append(w)

In [ ]:
values_wi = []
for w in W:
    if w in wit:
        values_wi.append(1)
    else:
        values_wi.append(0)

- $W = \text{ todas las palabras}$
- $Wt = \text{ palabras que esten en el texto}$
- $Wnt = \text{ palabras que no esten en el texto}$
- $W_i = 0 \text{ si } w_i \in wnt$
- $W_i = 1 \text{ si } w_i \in wt$

**Pregunta:**
$ P(A | W_0, W_1, \dots, W_n) $, donde se busca el valor más probable de $A$.

In [ ]:
CA, VA, p = QW_AT.Query(vars=(A,), indep=tuple([W[w] for w in W]), values_indep=tuple(values_wi))
VA, p

In [ ]:
Wt = []
Wt_v = []

for w in wit:
    Wt.append(W[w]),
    Wt_v.append(1)

In [ ]:
k = len(W) + 2 - len(Wt_v) 
print('10^',k)

In [ ]:
CA, VA, p = QW_AT.Query(vars=(A,), indep=tuple(Wt), values_indep=tuple(Wt_v))
VA, p

## Tests de rendimiento 

### Simulación

In [ ]:
import mib_v2_3 as mb2_3
import mib_v2_3_thr as mb2_3tr
from itertools import product
import time

time_Q_TAW_nv = []

for i in [10,11,12,13]:
    time_Q_TAW = []

    A = mb2_3.Var('A', set([j for j in range(10)]))
    T = mb2_3.Var('T', set([0,1,2]))
    W = [mb2_3.Var(j, set([0,1])) for j in range(i)]
    
    print("n_vocab: ",i)
    dA = {}
    for j in range(10):
        dA[(j,)] = 1/10
        
    PA = mb2_3.Distrib(dA, ('A'))
    
    
    dT_A = {}
    for j in range(10):
        dT_A[(j,)] = {}
        for k in range(3):
            dT_A[(j,)][(k,)] = 1/10
        
    PT_A = mb2_3.CondDistrib(dT_A, ('T',), ('A'))
    
    PW_AT = []
    for wi in W:
        dwi_TA = {}
        for par in product(*[[0,1,2], [j for j in range(10)]]):
            dwi_TA[par] = {}
            for j in range(2):
                dwi_TA[par][(j,)] = 0.5

        PWi_AT = mb2_3.CondDistrib(dwi_TA, (wi.getName(),), ('T','A'))
        PW_AT.append(PWi_AT)
    
    set_vars = set([A,T] + W)
    set_descomp = set([PA, PT_A,] + PW_AT)
    PATW = mb2_3.Specification(set_vars,set_descomp)
    
    QW_AT = mb2_3.Question(PATW)
    
    for j in range(i):
        Wt = [W[k-1] for k in range(j+1)]
        Wt_v = [1 for k in range(j+1)]
        
        inicio = time.time()
        CA, VA, p = QW_AT.Query(vars=(A,), indep=tuple(Wt), values_indep=tuple(Wt_v))
        fin = time.time()
        print("n_w:",j,"-",fin - inicio)
        time_Q_TAW.append(fin - inicio)
    
    time_Q_TAW_nv.append(time_Q_TAW)

In [ ]:
import matplotlib.pyplot as plt
nc = 2
nf = 2
fig, axs = plt.subplots(nc, nf)
fig.set_size_inches(10,8)
f = 0
c = 0
for i,t in enumerate(time_Q_TAW_nv):
    xn = [k+1 for k in range(len(t))]
    axs[c,f].plot(xn, t)
    axs[c,f].set_title("P(A|W0,...,Wr) con n = {n}".format(n = len(t)))

    axs[c,f].set_xlabel("r")
    axs[c,f].set_ylabel("segundos")
    
    if f == nf - 1:
        f = 0
        if c == nc - 1:
            c = 0
        else:
            c += 1
    else:
        f += 1

### Test de $P(A|W_0,W_1,\dots,W_n)$ y $P(A|W_0,W_1,\dots,W_r,W_{r+1},\dots,W_n)$

In [ ]:
import tpproc as tp
from collections import Counter  #regresa un diccionario con conteos
import glob
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from collections import OrderedDict #diccionarios ordenados
import numpy as np
import mib_v2_3 as mb

In [ ]:
archivos = glob.glob('./Train/*/*')
archivos,nombres = tp.carga_cuentos(archivos)
df_train = tp.lee_cuentos(archivos)
df_train.head()

In [ ]:
archivos = glob.glob('./Test/El Caballo y el Lobo.txt')
archivos,nombres = tp.carga_cuentos(archivos)
df_test = tp.lee_cuentos(archivos,test=True)
df_test.head()

In [ ]:
s = df_train[df_train['autor'] == 'jean de la fontaine'].texto.str.cat(sep=' ').split()
conteos = Counter(s)
autor_vocabulario = set([palabra for palabra in s])
print(len(autor_vocabulario))

In [ ]:
s = df_test.texto.str.cat(sep=' ').split()
conteos = Counter(s)
vocab_test = set([palabra for palabra in s])
print(len(vocab_test))

In [ ]:
df_train_T = df_train.copy()
df_train_T['nuevo_texto'] = df_train_T.texto.str.split().\
    apply(lambda texto: [w for w in texto if w in autor_vocabulario]).\
    apply(lambda x : ' '.join(x))
df_train_T['Conteos']=df_train_T.nuevo_texto.str.split().apply(Counter)
df_train_T.head()

In [ ]:
df_test_T = df_test.copy()
df_test_T['nuevo_texto']=df_test_T.texto.str.split().\
    apply(lambda texto: [w for w in texto if w in vocab_test]).\
    apply(lambda x : ' '.join(x))
df_test_T['Conteos']=df_test_T.nuevo_texto.str.split().apply(Counter)


df_test_T.head()

In [ ]:
from itertools import product
# Creación de variables

# Vraible para autor
# Ocurrencias de cada autor y conteos de ocurrencias y de número de autores
oc_autor = Counter(df_train_T.autor)
tot_oc_aut = np.sum(list(oc_autor.values()))
tot_autores = len(oc_autor)
A = mb.Var('A', set([autor for autor in oc_autor]))
# Distribución P(A)
n_= []
for autor in oc_autor:
    n_.append(((autor,),oc_autor[autor]/tot_oc_aut))

# Dicionario de valores de probabilidad
dA = dict(n_)
# Distribución de probabilidad
PA = mb.Distrib(vars=set([A]), table = dA, columns=('A',))
print(A.getValues())
PA.print_table()


# Variable para tipos
# Ocurrencias de cada tipo y conteos de ocurrencias y de número de tipos
oc_tipo = Counter(df_train_T.tipo)
tot_oc_tipo = np.sum(list(oc_tipo.values()))
tot_tipos = len(oc_tipo)
T = mb.Var('T', set([tipo for tipo in oc_tipo]))

# Distribución P(T|A)

# Ocurrencias (conteos) de cada combinación (tipo,autor)
conteo_pares = Counter(zip(df_train_T.autor,df_train_T.tipo))
# Combinaciones (tipo,autor)
autores = list(set(df_train_T.autor))
tipos = list(set(df_train_T.tipo))

dT_A = {}

for autor, tipo in product(*[autores,tipos]):
    par = (autor, tipo)
    ak = (autor,)
    
    if ak in dT_A.keys():
        if par in conteo_pares.keys():
            dT_A[ak][(tipo,)] = conteo_pares[par]
        else:
            dT_A[ak][(tipo,)] = 0 
    else:
        if par in conteo_pares.keys():
            dT_A[ak] = {(tipo,): conteo_pares[par]}
        else:
            dT_A[ak] = {(tipo,): 0}
            
# Corrección de Laplace en el caso general
# Se corrige la misma tabla que se senvía como argumento; no se crea una nueva.
def Laplace_gral(tabla):
    n = len(tabla[list(tabla.keys())[0]]) 
    for k in tabla.keys():
        registro = tabla[k]
        n_j = np.sum(list(registro.values()))
        for i in registro.keys():
            pb = (registro[i] + 1)/(n_j+n)
            tabla[k][i]=pb

Laplace_gral(dT_A)

PT_A = mb.CondDistrib(set([T]), set([A]), dT_A, (T.getName(),), (A.getName(),))
PT_A.print_table()

In [ ]:
vocab_inter = vocab_test & autor_vocabulario
vocab_dif = autor_vocabulario - vocab_test 

print(len(vocab_inter))
print(len(vocab_dif))

nuevo_vocab = vocab_inter.copy()

ps_rn = []
ps_n = []
xn_p = []

for i in range(20):
    xn_p.append(len(nuevo_vocab))
    df_train_T = df_train.copy()
    df_train_T['nuevo_texto'] = df_train_T.texto.str.split().\
        apply(lambda texto: [w for w in texto if w in nuevo_vocab]).\
        apply(lambda x : ' '.join(x))
    df_train_T['Conteos']=df_train_T.nuevo_texto.str.split().apply(Counter)
    
    # Palabras
    W = {}
    for w in nuevo_vocab:    # vocabulario reducido
        W[w] = mb.Var(w,set([0,1]))
    
    # Combinaciones (tipo,autor)
    autores = list(set(df_train_T.autor))
    tipos = list(set(df_train_T.tipo))
    pares_ta = list(product(autores,tipos))

    # Conteos de palabras en nuevo_texto por pares (a,t) en el data frame de train
    conteo_w = dict(df_train_T.Conteos)
    pares_train = list(zip(df_train_T.autor,df_train_T.tipo))

    for i,k in enumerate(conteo_w.keys()):
        conteo_w[k] = {pares_train[i] : dict(conteo_w[k])}
        
    def check(palabra,pares_ta):
        pw_ = {}
        var = W[palabra]
        
        for par in pares_ta:
            cw_1 = 0   # conteos de presencia 
            cw_0 = 0   # y ausencia en 0
            it = 0     # total de pares (a,t) contabilizados
            
            for item in list(conteo_w.values()):  # checamos cada par (a,t) en el conjunto de train
                # print(i,list(item.keys())[0], end=' ')
                it += 1
                if par in item: # checamos si el par (a,t) de entrada tiene correspondencia en Train
                    if palabra in list(item.values())[0]: #checamos si la palabra existe en el par (a,t) de Train
                        cw_1 += 1
                    else:
                        cw_0 += 1  #debemos contar también la no existencia por si hay igualdad
            
            if cw_0 == cw_1: # la palabra aparece por igual en todos los pares Train, o bien, no hay pares Train
                pw_1 = 0.5   #corrección de Laplace que indica que la palabra puede o no estar por igual
                pw_0 = 0.5
                
            else:
                pw_1=(1+cw_1)/(len(var.getValues())+it) #corrección de Laplace en caso de que la palabra exista
                pw_0 = 1-pw_1
            
            pw_[par] = {(0,): pw_0, (1,): pw_1}
        pw_ = OrderedDict(sorted(pw_.items()))
        return pw_
            
        
    PW_AT = {}
    for w in nuevo_vocab:    # vocabulario reducido
        t = check(w,pares_ta)
        PW_AT[w] = mb.CondDistrib(set([W[w]]),set([A,T]), dict(t), (W[w].getName(),), (A.getName(), T.getName()))
        
    vars_set = set([A,T] + [W[w] for w in W]) 
    descomp_set = set([PA, PT_A] + [PW_AT[w] for w in W])
    PATW = mb.Specification(vars_set, descomp_set)
    
    Q_PATW = mb.Question(PATW)
    
    values_wi = []
    for w in W:
        if w in vocab_inter:
            values_wi.append(1)
        else:
            values_wi.append(0)
    
    p = Q_PATW.Query(vars=tuple([A]), indep=tuple([W[w] for w in W]), values_var = tuple(['jean de la fontaine']),values_indep = values_wi)
    ps_n.append(p)
    
    indep_wi = tuple([W[w] for w in vocab_inter])
    values_iwi = tuple([1 for w in indep_wi])
    p = Q_PATW.Query(vars=tuple([A]), indep = indep_wi, values_var = tuple(['jean de la fontaine']),values_indep = values_iwi)
    ps_rn.append(p)
    
    nuevo_vocab.add(vocab_dif.pop())
    

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1, 2)
fig.set_size_inches(15,5)

axs[0].plot(xn_p, ps_n, color = 'r',label = "Como decisión")
axs[0].set_title("P(A|W0,W1,Wr,...,Wn)")
axs[0].set_xlabel("n")
axs[0].set_ylabel("P")


axs[1].plot([n-20 for n in xn_p], ps_rn, color = 'b', label = "Con división")
axs[1].set_title("P(A|W0,W1,Wr)")
axs[1].set_xlabel("n - r")
axs[1].set_ylabel("P")
plt.show()